In [79]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from crewai import Agent
from crewai_tools import SerperDevTool
from crewai import Task
from crewai import Crew, Process

In [80]:
# Load environment variables from .env file
load_dotenv()

True

In [81]:
# Access the environment variables
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GROQ_API_BASE = os.getenv("GROQ_API_BASE") 
GROQ_MODEL = os.getenv("GROQ_MODEL", "groq/llama-3.3-70b-versatile")


sender_email = os.getenv("SENDER_EMAIL")
sender_password = os.getenv("SENDER_PASSWORD")
recipient_email = os.getenv("RECIPIENT_EMAIL")

In [82]:
search_tool = SerperDevTool()

In [83]:
import os
from crewai import LLM

groq_llm = LLM(
        model = GROQ_MODEL,
        api_key = GROQ_API_KEY,
        base_url = GROQ_API_BASE,
        temperature=0.3, 
        max_tokens=512)

llm = groq_llm

In [84]:
# Creating an insight_analyst agent with memory and verbose mode
#insight_analyst = Agent(
    #role="Insight Analyst",
    #backstory=("""A skilled analyst trained to extract meaning from data and research findings."""),
    #verbose=True, 
    #memory = True, 
    #tools=[search_tool],
    #allow_delegation=True, 
    #llm= llm,
#)

In [85]:
# Creating an reporter agent with memory and verbose mode
reporter = Agent(
    role="Reporter",
    goal="Create detailed reports based on insight from research findings",
    backstory= """With a flair for simplifying complex topics, you craft
        engaging narratives that captivate and educate, bringing new
        discoveries to light in an accessible manner.""",
    verbose=True, 
    memory = True, 
    #tools=[search_tool],
    allow_delegation=True, 
    llm= llm,
)

In [86]:
# Creating a senior researcher agent with memory and verbose mode
topic_researcher = Agent(
    role='Researcher',
    goal=' Uncover cutting-edge developments in {topic}',
    verbose=True,
    memory=False,
    backstory=(
       "Expert in uncovering current developments, with a focus on relevance and clarity."
    ),
    tools=[search_tool],
    allow_delegation=False,
    llm= llm,
)

In [87]:
critic = Agent(
    role="Compliance Critic",
    goal="Ensure that research output adheres to legal, ethical, and factual standards",
    backstory="Responsible for analyzing content for misinformation, copyright issues, and ethical violations.",
    verbose=True,
    memory = True,
    tools=[search_tool],
    allow_delegation=False, 
    llm= llm,
) 

In [88]:
#analyze_task = Task(
   #description= "Analyze the raw research report and extract patterns, trends, and important data.", 
   #expected_output="Key insights and conclusions from the report.",
   #agent= insight_analyst,
   #tools=[search_tool], 
   #context = [research_task]
#)

In [89]:
reporter_task = Task( 
    description= "Convert the research findings into a readable report. Break down into clear sections by subtopic.",
    expected_output= "A well-structured report in markdown format, organized by topic.",
    agent= reporter,
    #tools=[search_tool], 
    context = [research_task],
)

In [90]:
research_task = Task(
    description= "Conduct web research on: '{topic}' and gather relevant data.",
    expected_output="A concise research summary report with key points and source links.",
    tools=[search_tool],
    agent=topic_researcher,
)

In [91]:
critic_task = Task( 
    description="Evaluate the summary for compliance with legal, ethical, and factual standards. "
                "Ensure no false claims, offensive language, or unverified info is present.",
    expected_output="A compliance-checked version of the summary with necessary corrections.",
    agent=critic,
    tools=[search_tool],
    context=[reporter_task],
    output_file= "final_compliance_checked.md" 
 )

In [92]:
# Task: Reporter revises based on critic feedback
revision_task = Task(
    description="Revise the report using the feedback provided by the critic. Apply all suggested corrections while maintaining clarity and accuracy.",
    expected_output="A final report that incorporates all feedback and is ready for publishing.",
    agent=reporter,
    #tools=[search_tool],
    context=[critic_task],  # feeds critic’s feedback back to reporter
    output_file="final-report.md"
)


In [93]:
# Forming the crew with enhanced configurations
crew = Crew(
    agents=[reporter, topic_researcher, critic],
    tasks=[reporter_task, research_task, critic_task, revision_task],
    process=Process.sequential 
)

# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'The role of AI in improving food security in Africa'})
print(result)

# Agent: Reporter
## Task: Convert the research findings into a readable report. Break down into clear sections by subtopic.
# Agent: Researcher
## Task: Gather all research findings


# Agent: Researcher
## Thought: Thought: To gather all research findings on the role of AI in improving food security in Africa, I need to search the internet for relevant information. I will use the tool "Search the internet with Serper" to find the latest research and developments in this area.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"AI in food security Africa research findings\"}"
## Tool Output: 
{'searchParameters': {'q': 'AI in food security Africa research findings', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': "Enhancing Africa's agriculture and food systems through ... - PubMed", 'link': 'https://pubmed.ncbi.nlm.nih.gov/39917548/', 'snippet': 'The integration of artificial intelligence (AI) technologies into agriculture hold

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxsdg1trfmatwpzz24z0819k` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Used 9738, Requested 2578. Please try again in 1.579s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
